### Черновик для собеседования по теме сверточного перемежения/деперемежения. Язык Julia.
### Содержит части - Реализацию самих функции, Примеры использования,Тесты и Пояснительная часть.
### Можно выполнять сверху вниз, либо в любой последовательности после выполнения части 1. 

## 1.Функции сверточного перемежения/деперемежения

In [1]:

function convolutional_interleave(input_sequence::Vector{Q}, num_registers::Int=6, register_length_step::Int=2,initial_conditions=undef)::Vector{Q} where Q
# функция сверточного перемежения. https://www.mathworks.com/help/comm/ref/comm.convolutionalinterleaver-system-object.html 
# input_sequence - вектор входной последовательности, Тип значений Int, Float, Char
# num_registers - число сдвиговых регистров преобразователя (Int)
# register_length_step - задает задержку в регистрах
# initial_conditions - начальные состояния ячеек сдвиговых регистров. Варианты:
#       (не указан) - регистры инициализируются 0 для входных значений Int, 0.0 для Float. (для Char не реализован)
#       (задано одно число) - регистры инициализируются этим числом  (для Char не реализован)
#       (задан массив ячеек регистров в явном виде) - регисты инициализируются как задано,
#         + игнорируются явно заданные num_registers и register_length_step
    output = Vector{Q}(undef, length(input_sequence))

    if initial_conditions==undef
        # создаем регистры необходимой длины, заполняем нулями
        shift_registers = [zeros(Q, (i-1) * register_length_step +1) for i in 1:num_registers]
    elseif length(initial_conditions)==1
        # создаем регистры необходимой длины и заполняем аргументом
        shift_registers = [ones(Q, (i-1) * register_length_step +1) for i in 1:num_registers]
        shift_registers = shift_registers*initial_conditions[1]
    else
        # берем значения регистров, переданные в качестве аргумента
        shift_registers = initial_conditions                
        num_registers = length(shift_registers) # число регистров берем фактически переданное
    end
    
    # идем циклом по всей входной последовательности
    for i in 1:length(input_sequence)
        symbol = input_sequence[i]
        reg_i = 1+(i+1)%num_registers #индекс регистра, куда пишем
        shift_registers[reg_i] = circshift(shift_registers[reg_i], 1) # двигаем регистр 
        shift_registers[reg_i][1] = symbol # пишем новое значение в регистр
        register_result = shift_registers[reg_i][end] # значение, которое выплюнул выход регистра
        output[i] = register_result
        end
    
    return output
end

function convolutional_deinterleave(input_sequence::Vector{Q}, num_registers::Int=6, register_length_step::Int=2,initial_conditions=undef)::Vector{Q} where Q
# функция сверточного деперемежения. https://www.mathworks.com/help/comm/ref/comm.convolutionalinterleaver-system-object.html 
# input_sequence - вектор входной последовательности, Тип значений Int, Float, Char
# num_registers - число сдвиговых регистров преобразователя (Int)
# register_length_step - задает задержку в регистрах
# initial_conditions - начальные состояния ячеек сдвиговых регистров. Варианты:
#       (не указан) - регистры инициализируются 0 для входных значений Int, 0.0 для Float. (для Char не реализован)
#       (задано одно число) - регистры инициализируются этим числом  (для Char не реализован)
#       (задан массив ячеек регистров в явном виде) - регисты инициализируются как задано,
#         + игнорируются явно заданные num_registers и register_length_step
    
    output = Vector{Q}(undef, length(input_sequence))

    if initial_conditions==undef
        # создаем регистры необходимой длины, заполняем нулями
        shift_registers = [zeros(Q, (i-1) * register_length_step +1) for i in num_registers:-1:1]
    elseif length(initial_conditions)==1
        # создаем регистры необходимой длины заполняем аргументом
        shift_registers = [ones(Q, (i-1) * register_length_step +1) for i in num_registers:-1:1]
        shift_registers = shift_registers*initial_conditions[1]
    else
        # берем значения регистров, переданные в качестве аргумента
        shift_registers = initial_conditions                
        num_registers = length(shift_registers) # число регистров берем фактически переданное
    end
    
    # идем циклом по всей входной последовательности
    for i in 1:length(input_sequence)
        symbol = input_sequence[i]
        reg_i = 1+(i+1)%num_registers #индекс регистра, куда пишем
        shift_registers[reg_i] = circshift(shift_registers[reg_i], 1) # двигаем регистр 
        shift_registers[reg_i][1] = symbol # пишем новое значение в регистр
        register_result = shift_registers[reg_i][end] # значение, которое выплюнул выход регистра
        output[i] = register_result
        end
    
    return output
end


convolutional_deinterleave (generic function with 4 methods)

## 2.Примеры использования

In [ ]:
# пример 1 - сверточное прореживание, параметры как в документации 
# https://www.mathworks.com/help/comm/ref/comm.convolutionalinterleaver-system-object.html
# число регистров num_registers = 2, шаг регистров register_length_step = 3, инициализация 0(умолчание)
data1 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] 
num_registers = 2
register_length_step = 3
data2 = convolutional_interleave(data1,num_registers,register_length_step)  
data3 = convolutional_deinterleave(data2,num_registers,register_length_step)
for i in 1:length(data1)
    println(data1[i],"\t",data2[i],"\t",data3[i])
end


In [ ]:
# пример 2 - сверточное прореживание, параметры 
# число регистров num_registers = 2, шаг регистров register_length_step = 3,
# инициализация initial_conditions = 100 
data1 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
data2 = convolutional_interleave(data1,2,3,100)
data3 = convolutional_deinterleave(data2,2,3,100)
for i in 1:length(data1)
    println(data1[i],"\t",data2[i],"\t",data3[i])
end


In [ ]:
# пример 3 - сверточное прореживание, параметры 
# число регистров num_registers = 2 (игнорируется), шаг регистров register_length_step = 3(игнорируется),
# инициализация регистров initial_conditions=[[100],[200,300,400,500]] - размерность (2х3), задана явно
# !! при явном задании значений регистров initial_conditions в виде векторов параметры функции "число регистров и шаг регистров вычисляются из нее !!
data1 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
data2 = convolutional_interleave(data1,2,3,[[10],[200,300,400,500]])
data3 = convolutional_deinterleave(data2,2,3)
for i in 1:length(data1)
    println(data1[i],"\t",data2[i],"\t",data3[i])
end


In [ ]:
# пример 4 - сверточное прореживание для символьной последовательности, параметры 
# число регистров num_registers = 2 (игнорируется), шаг регистров register_length_step = 3(игнорируется),
# инициализация регистров initial_conditions=[['_'],['_','_','_','_']] - размерность (2х3), задана явно
# !! при явном задании значений регистров initial_conditions в виде векторов параметры функции "число регистров и шаг регистров вычисляются из нее !!

data1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
data2 = convolutional_interleave(data1,2,3,[['_'],['_','_','_','_']])
data3 = convolutional_interleave(data2,2,2,[['_','_','_','_'],['_']])

for i in 1:length(data1)
    println(data1[i],"\t",data2[i],"\t",data3[i])
    end

## 3.Тесты для кода

In [ ]:
# тесты для функций сверточного перемежения и деперемежения
# test_1 - test_6 проверяют численные ряды из документации https://www.mathworks.com/help/comm/ref/comm.convolutionalinterleaver-system-object.html
# test_7 - test_9 проверяют задержки Delays of Convolutional Interleaving and Deinterleaving из того же документа

function test_1()
    data1 = [0,1,2,3,4,5,6,7,8,9]
    data2 = convolutional_interleave(data1,2,3)
    data3 = [0, 0, 2, 0, 4, 0, 6, 1, 8, 3]
    if data2==data3
        println("test_1 success")
        return true
    else
        println("test_1 failed")
        return false
    end
end

function test_2()
    data1 = [0, 0, 2, 0, 4, 0, 6, 1, 8, 3]
    data2 = convolutional_deinterleave(data1,2,3)
    data3 = [0, 0, 0, 0, 0, 0, 0, 1, 2, 3]
    if data2==data3
        println("test_2 success")
        return true
    else
        println("test_2 failed")
        return false
    end
end

function test_3()
    data1 = [0,1,2,2,4,5,6,7,8,9]
    data2 = convolutional_interleave(data1,2,3)
    data3 = [0, 0, 2, 0, 4, 0, 6, 1, 8, 3]
    if data2!=data3
        println("test_3 success")
        return true
    else
        println("test_3 failed")
        return false
    end
end

function test_4()
    data1 = [0, 0, 0, 2, 0, 4, 0, 6, 1, 8, 3]
    data2 = convolutional_deinterleave(data1,2,3)
    data3 = [0, 0, 0, 0, 0, 0, 0, 1, 2, 3]
    if data2!=data3
        println("test_4 success")
        return true
    else
        println("test_4 failed")
        return false
    end
end

function test_5()
    data1 = [0,1,2,3,4,5,6,7,8,9]
    data2 = convolutional_interleave(data1,3,2)
    data3 = [0, 0, 2, 0, 4, 0, 6, 1, 8, 3]
    if data2!=data3
        println("test_5 success")
        return true
    else
        println("test_5 failed")
        return false
    end
end

function test_6()
    data1 = [0, 0, 2, 0, 4, 0, 6, 1, 8, 3]
    data2 = convolutional_deinterleave(data1,3,2)
    data3 = [0, 0, 0, 0, 0, 0, 0, 1, 2, 3]
    if data2!=data3
        println("test_6 success")
        return true
    else
        println("test_6 failed")
        return false
    end
end

function test_7()
    NumRegisters = 4
    RegisterLengthStep = 3

    data1 = [0,2,5,2,7,4,9,5,3,6,8,2,1,12,5,23,7,45,34,23,98,0,0,3,2,6,5,4,3,7,1,2,3,4,5,6,9,8,7,6,5,4,3]
    
    data2 = convolutional_interleave(data1,NumRegisters,RegisterLengthStep)
    data3 = convolutional_deinterleave(data2,NumRegisters,RegisterLengthStep);
    
    totalDelay = NumRegisters*RegisterLengthStep*(NumRegisters-1)
    #println("totalDelay ",totalDelay)

    for i in 1:(length(data1)-totalDelay)
        #print(data1[i],"\t",data2[i],"\t",data3[i+totalDelay])
        #println();
        if data1[i]!=data3[i+totalDelay]
            println("test_7 failed")
            return false
            end
        end 
    
    println("test_7 success")
    return true
end

function test_8()
    NumRegisters = 2 
    RegisterLengthStep = 3 

    data1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    data2 = convolutional_interleave(data1,NumRegisters,RegisterLengthStep,[[' '],[' ',' ',' ',' ']])
    data3 = convolutional_deinterleave(data2,NumRegisters,RegisterLengthStep,[[' ',' ',' ',' '],[' ']])

    totalDelay = NumRegisters*RegisterLengthStep*(NumRegisters-1)
    #println("totalDelay ",totalDelay)

    for i in 1:(length(data1)-totalDelay)
        #print(data1[i],"\t",data2[i],"\t",data3[i+totalDelay])
        #println();
        if data1[i]!=data3[i+totalDelay]
            println("test_8 failed")
            return false
            end
        end 
    
    println("test_8 success")
    return true
end

function test_9()
    NumRegisters = 2 
    RegisterLengthStep = 2 

    data1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    data2 = convolutional_interleave(data1,NumRegisters,RegisterLengthStep,[[' '],[' ',' ',' ']])
    data3 = convolutional_deinterleave(data2,NumRegisters,RegisterLengthStep,[[' ',' ',' '],[' ']])

    totalDelay = NumRegisters*RegisterLengthStep*(NumRegisters-1)
    #println("totalDelay ",totalDelay)

    for i in 1:(length(data1)-totalDelay)
        #print(data1[i],"\t",data2[i],"\t",data3[i+totalDelay])
        #println();
        if data1[i]!=data3[i+totalDelay]
            println("test_9 failed")
            return false
            end
        end 
    
    println("test_9 success")
    return true
end


function test_all()
    test_1()
    test_2()
    test_3()
    test_4()
    test_5()
    test_6()
    test_7()
    test_8()
    test_9()
end

test_all()

## 4.Пояснительная часть

### Возможно странный код -
Первый код на Julia, пытался писать максимально просто, не используя вещей, в которых нет времени досконально разобраться. Практикующего программиста Julia ими не удивить, а код новичка в языке они не сделают лучше.
### Почему такие странные тесты -
Для тестовых данных использованы числовые ряды и закон формирования задержек из https://www.mathworks.com/help/comm/ref/comm.convolutionalinterleaver-system-object.html
MatlabOnline на моей рабочей машине не заработал (не проходит регистрация, санкции?) - поэтому данные только из документов. Тесты писаны врукопашную максимально просто. Добавил бы еще тесты для разных размерностей, при нормальном доступе к эталону.  
### Оптимизация - 
Оптимизацию не делал, как и измерения производительности. Оптимизация возможна.
### Блок перемежения/деперемежения или функции? - 
Для реализации в виде блока в стиле matlab необходимо между вызовами функций где-то отдельно хранить конечные состояния сдвиговых регистров и индекс последней записи в регистр и передавать их в вычислитель при последующем вызове, т.е. хранить их промежуточные состояния. Технически - создать объект для реализации функционала и хранения промежуточных данных. Либо из функции возвращать кортежем эти состояния для внешнего промежуточного хранения. Вариантов реализации много, не уверен что это нужно в данном конкретном случае.     
